# Google API i geokodowania

[Piotr Migdał](http://migdal.wikidot.com/), DELab UW

Będziemy geokodować adresy przy pomocy  [The Google Geocoding API](https://developers.google.com/maps/documentation/geocoding/).

### Słowniczek

* [Geokodowanie](https://pl.wikipedia.org/wiki/Geokodowanie) - ustalanie współrzędnych geograficznych na podstawie innych danych geograficznych (np. kodu pocztowego, ulicy i miasta). 
* [API](https://pl.wikipedia.org/wiki/Application_Programming_Interface) (Application Programming Interface) - ogólne słowo sposób komunikacji z programem, zwłaszcza: zdalnym
* [JSON](https://pl.wikipedia.org/wiki/JSON) (JavaScript Object Notation) - tekstowy sposób zapisywania danych, który pozwala zapisać różne dane, nie tylko tabele (jak CSV)
* [XML](https://en.wikipedia.org/wiki/XML) (Extensible Markup Language) 

### Zapytania

* https://maps.googleapis.com/maps/api/geocode/json?address=ul.+Dobra+56/66,+Warszawa
    * dobra lokalizacja? sprawdzić możemy np. tak:
    https://www.google.com/maps/@52.2423917,21.0244245,16z
* https://maps.googleapis.com/maps/api/timezone/json?location=39.6034810,-119.6822510&timestamp=1331161200
* https://maps.googleapis.com/maps/api/elevation/json?locations=39.7391536,-104.9847034
* https://maps.googleapis.com/maps/api/directions/json?origin=Warszawa&destination=Kraków

Spróbuj:

* Zastanowić co się w nich pyta, i co oznacza odpowiedź.
* Zadać inne pytania.
* Poprosić o odpowiedź w formacie XML (spójrz na podobieństwa i różnice).

### Ograniczenia

Wersja darmowa Google API pozwala na 2500 zapytań w ciągu 24h, i nie więcej niż 5 na sekundę.

### Inne przykładowe API

* http://api.worldbank.org/countries?format=json
* https://en.wikipedia.org/w/api.php?action=query&list=backlinks&bltitle=Almond&bllimit=500&blfilterredir=nonredirects&format=json
* https://api.stackexchange.com/2.2/tags/python/related?site=stackoverflow

### Python

Surowa forma pytań i odpowiedzi jest celowa - ułatwia przetwarzanie przez komputer (dla niego łatwije wysłać `"json?location=39.6034810,-119.6822510&timestamp=1331161200"` niż wyklikać na stronie internetowej).


In [1]:
# biblioteka do zapytań 
import requests

In [2]:
r = requests.get("https://maps.googleapis.com/maps/api/elevation/json?locations=39.7391536,-104.9847034")

In [3]:
# surowa treść odpowiedzi
r.content

b'{\n   "results" : [\n      {\n         "elevation" : 1608.637939453125,\n         "location" : {\n            "lat" : 39.7391536,\n            "lng" : -104.9847034\n         },\n         "resolution" : 4.771975994110107\n      }\n   ],\n   "status" : "OK"\n}\n'

In [4]:
# wczytanie JSONa
odpowiedz = r.json()
odpowiedz

{'results': [{'elevation': 1608.637939453125,
   'location': {'lat': 39.7391536, 'lng': -104.9847034},
   'resolution': 4.771975994110107}],
 'status': 'OK'}

In [5]:
# dostaliśmy słownik, w którym są inne obiekty
# np. napis
odpowiedz['status']

'OK'

In [6]:
# np. lista
odpowiedz['results']

[{'elevation': 1608.637939453125,
  'location': {'lat': 39.7391536, 'lng': -104.9847034},
  'resolution': 4.771975994110107}]

In [7]:
# jeśli nie wiemy jakiego typu jest dany obiekt, używamy `type`
type(odpowiedz['results'])

list

In [8]:
# by dostać co nas interesuje zwykle musimy kilkukrotnie wchodził wgłąb
odpowiedz['results'][0]['location']['lng']

-104.9847034

In [9]:
# innymi przydatnymi narzędami jest sprawdzenie długości listy
len(odpowiedz['results'])

1

In [10]:
# oraz sprawdzenie kluczy 
odpowiedz['results'][0].keys()

dict_keys(['elevation', 'location', 'resolution'])

## Przekazywanie parametrów

Typowe zapytania mają parametry. Np.

* https://maps.googleapis.com/maps/api/geocode/json?address=ul.+Dobra+56/66,+Warszawa

składa się z

* głównego adresu `https://maps.googleapis.com/maps/api/geocode/json`,
* parametru `address` równemu `Warszawa`.

Zapytania mają postać `glowny_adres?parametr1=wartosc1&parametr2=wartosc2&parametr3=wartosc3` itd. Zwykle kolejność parametrów nie ma znaczenia.

Do zapytań można "ręcznie" tworzyć pełen adres, lub generować parametry z pythonowych słowników (druga opcja jest zwkle dużo przyjemniejsza).

In [11]:
r = requests.get("https://maps.googleapis.com/maps/api/geocode/json",
                 params={"address": "ul. Dobra 56/66, Warszawa"})

In [12]:
# pobierze nam to dane z wygenerowanym adresem:
print(r.url)

https://maps.googleapis.com/maps/api/geocode/json?address=ul.+Dobra+56%2F66%2C+Warszawa


In [13]:
# długość odpowiedzi
len(r.content)

1989

In [14]:
odpowiedz = r.json()
odpowiedz

{'results': [{'address_components': [{'long_name': '56/66',
     'short_name': '56/66',
     'types': ['street_number']},
    {'long_name': 'Dobra', 'short_name': 'Dobra', 'types': ['route']},
    {'long_name': 'Śródmieście',
     'short_name': 'Śródmieście',
     'types': ['sublocality_level_1', 'sublocality', 'political']},
    {'long_name': 'Warszawa',
     'short_name': 'Warszawa',
     'types': ['locality', 'political']},
    {'long_name': 'Warszawa',
     'short_name': 'Warszawa',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'mazowieckie',
     'short_name': 'mazowieckie',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'Poland',
     'short_name': 'PL',
     'types': ['country', 'political']}],
   'formatted_address': 'Dobra 56/66, Warszawa, Poland',
   'geometry': {'location': {'lat': 52.2423917, 'lng': 21.0244245},
    'location_type': 'ROOFTOP',
    'viewport': {'northeast': {'lat': 52.2437406802915,
      'lng'

Widać, że dostajemy dużo informacji. Jak się po tym poruszać?

In [15]:
# klucze słownika
odpowiedz.keys()

dict_keys(['status', 'results'])

In [16]:
# typ elementu czyli czy to:
# napis `str`, lista `list', słownik `dict` lub liczba `int` lub `float`
type(odpowiedz["status"])

str

In [17]:
odpowiedz["status"]

'OK'

In [18]:
type(odpowiedz["results"])

list

In [19]:
len(odpowiedz["results"])

1

In [20]:
odpowiedz["results"][0].keys()

dict_keys(['formatted_address', 'place_id', 'address_components', 'types', 'geometry', 'partial_match'])

In [21]:
# i tak do interesujących nas rezultatów
odpowiedz["results"][0]["geometry"]["location"]

{'lat': 52.2423917, 'lng': 21.0244245}

## Co jeszcze możemy z nimi zrobić?

Zapisać i wczytać!

In [22]:
# można też wczytywać i zapisywać takie dane na dysku; mają format json
import json

In [23]:
odpowiedz

{'results': [{'address_components': [{'long_name': '56/66',
     'short_name': '56/66',
     'types': ['street_number']},
    {'long_name': 'Dobra', 'short_name': 'Dobra', 'types': ['route']},
    {'long_name': 'Śródmieście',
     'short_name': 'Śródmieście',
     'types': ['sublocality_level_1', 'sublocality', 'political']},
    {'long_name': 'Warszawa',
     'short_name': 'Warszawa',
     'types': ['locality', 'political']},
    {'long_name': 'Warszawa',
     'short_name': 'Warszawa',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'mazowieckie',
     'short_name': 'mazowieckie',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'Poland',
     'short_name': 'PL',
     'types': ['country', 'political']}],
   'formatted_address': 'Dobra 56/66, Warszawa, Poland',
   'geometry': {'location': {'lat': 52.2423917, 'lng': 21.0244245},
    'location_type': 'ROOFTOP',
    'viewport': {'northeast': {'lat': 52.2437406802915,
      'lng'

In [24]:
# tak zapisujemy plik JSON (folder musi istnieć)
# open(sciezka, 'w') znaczy, że otwieramy plik do zapisu
json.dump(odpowiedz, open("../dane/gdzie_buw.json", "w"), indent=2)
# teraz warto go otworzyć

Warto zobaczyć jak wygląda w edytorze tekstu, np. [Sublime Text](http://www.sublimetext.com/) (płatny, ale można długo próbować) lub [Notepad++](https://notepad-plus-plus.org/) (darmowy, Windows). Do mniejszych plików (tak do 2MB) jest [Atom](https://atom.io/). Jak chcesz głęboką wodę, to można spróbować Vim lub Emacs.

In [25]:
# a tak wczytujemy
zaladowany = json.load(open("../dane/gdzie_buw.json"))

In [26]:
zaladowany

{'results': [{'address_components': [{'long_name': '56/66',
     'short_name': '56/66',
     'types': ['street_number']},
    {'long_name': 'Dobra', 'short_name': 'Dobra', 'types': ['route']},
    {'long_name': 'Śródmieście',
     'short_name': 'Śródmieście',
     'types': ['sublocality_level_1', 'sublocality', 'political']},
    {'long_name': 'Warszawa',
     'short_name': 'Warszawa',
     'types': ['locality', 'political']},
    {'long_name': 'Warszawa',
     'short_name': 'Warszawa',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'mazowieckie',
     'short_name': 'mazowieckie',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'Poland',
     'short_name': 'PL',
     'types': ['country', 'political']}],
   'formatted_address': 'Dobra 56/66, Warszawa, Poland',
   'geometry': {'location': {'lat': 52.2423917, 'lng': 21.0244245},
    'location_type': 'ROOFTOP',
    'viewport': {'northeast': {'lat': 52.2437406802915,
      'lng'

Jeśli chcemy używać czegoś wielokrotnie, warto zrobić z tego funkcję.

In [27]:
# https://maps.googleapis.com/maps/api/elevation/json?locations=39.7391536,-104.9847034
def wysokosc(szer, dlug):
    r = requests.get("https://maps.googleapis.com/maps/api/elevation/json",
                     params={"locations": str(szer) + "," +  str(dlug)})
    odpowiedz = r.json()
    if len(odpowiedz['results']) > 0:
        return odpowiedz['results'][0]['elevation']
    else:
        return {}

In [28]:
# jaka jest Mt Everest (uwaga - współrzędne nie są przybliżone, więc możemy trafić koło szczytu)
wysokosc(27 + 59/60 + 17/60**2, 86 + 55/60 + 31/60**2)

8815.705078125

In [29]:
def lokalizuj(adres):
    r = requests.get("https://maps.googleapis.com/maps/api/geocode/json",
                     params={"address": adres}).json()
    return r['results'][0]['geometry']['location']

In [30]:
lokalizuj("Mount Everest")

{'lat': 27.9879017, 'lng': 86.9253141}

In [31]:
lokalizuj("ul. Polna, Warszawa")

{'lat': 52.2167206, 'lng': 21.0169635}

In [32]:
# podglądając jak Google Maps koduje pozycję w adresie
# https://www.google.com/maps/@52.2423917,21.0244245,16z
def lok2link(lokalizacja):
    return "https://www.google.com/maps/@" + str(lokalizacja['lat']) + "," + str(lokalizacja['lng']) + ",16z"

In [33]:
print(lok2link({'lat': 52.2167206, 'lng': 21.0169635}))

https://www.google.com/maps/@52.2167206,21.0169635,16z


In [34]:
print(lok2link(lokalizuj("Zabrze")))

https://www.google.com/maps/@50.3249278,18.7857186,16z


Do rzeczy! Zlokalizujmy szkoły w Warszawie.

Dane pochodzą z Zespołu Pomiaru Dydaktycznego Instytutu Badań Edukacyjnych, przez [pakiet R zozlak/ZDP](https://github.com/zozlak/ZPD), następnie przetworzony w ramach [badania wyników matur w DELab](https://github.com/stared/delab-matury).

In [35]:
import pandas as pd

In [36]:
licea = pd.read_csv("../dane/licea_warszawskie.csv")

In [37]:
len(licea)

165

In [38]:
licea.head()

,id_szkoly,nazwa_szkoly,adres,miejscowosc,kod_pocztowy,dzielnica
0,25302,Liceum Ogólnokształcące dla Dorosłych nr 48 MGM,Obrońców Tobruku 40,Warszawa,01-494,Bemowo
1,25300,Sportowe Liceum Ogólnokształcące nr 67 MGM,Obrońców Tobruku 40,Warszawa,01-494,Bemowo
2,25298,LXXVIII Liceum Ogólnokształcące im. Marii Pawl...,Anieli Krzywoń 3,Warszawa,01-391,Bemowo
3,25499,Liceum Ogólnokształcące Niepubliczne nr 29 im....,Powązkowska 90,Warszawa,01-728,Bemowo
4,25303,CV Liceum Ogólnokształcące im. Zbigniewa Herberta,Vincenta van Gogha 1,Warszawa,03-188,Białołęka


In [39]:
licea["dzielnica"].value_counts()

Śródmieście       33
Mokotów           23
Praga-Południe    17
Wola              16
Bielany           13
Ursynów           12
Praga-Północ       9
Ochota             8
Żoliborz           7
Wawer              7
Targówek           6
Bemowo             4
Włochy             2
Białołęka          2
Wesoła             2
Wilanów            2
Rembertów          1
Ursus              1
dtype: int64

In [40]:
# listę adresów dostejemy tak
list(licea["adres"])[:5]

['Obrońców Tobruku 40',
 'Obrońców Tobruku 40',
 'Anieli Krzywoń 3',
 'Powązkowska 90',
 'Vincenta van Gogha 1']

### Zadanie

Dla wybranej przez siebie dzielnicy Warszawy dodać kolumny `szerokosc` i `dlugosc` zgodnie z geolokalizacją szkoły.

## Zobacz też

* [San Francisco's Drug Geography](http://lmart999.github.io/2015/02/28/gis/)